In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate

# === Load synthetic ultrasonic data ===
temporal_df = pd.read_csv('ultrasonic_temporal.csv')
spatial_df = pd.read_csv('ultrasonic_spatial.csv')

In [2]:
print(temporal_df.columns.tolist())


['Segment_ID', 't0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24', 't25', 't26', 't27', 't28', 't29', 't30', 't31', 't32', 't33', 't34', 't35', 't36', 't37', 't38', 't39', 't40', 't41', 't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't50', 't51', 't52', 't53', 't54', 't55', 't56', 't57', 't58', 't59', 't60', 't61', 't62', 't63', 't64', 't65', 't66', 't67', 't68', 't69', 't70', 't71', 't72', 't73', 't74', 't75', 't76', 't77', 't78', 't79', 't80', 't81', 't82', 't83', 't84', 't85', 't86', 't87', 't88', 't89', 't90', 't91', 't92', 't93', 't94', 't95', 't96', 't97', 't98', 't99', 't100', 't101', 't102', 't103', 't104', 't105', 't106', 't107', 't108', 't109', 't110', 't111', 't112', 't113', 't114', 't115', 't116', 't117', 't118', 't119', 't120', 't121', 't122', 't123', 't124', 't125', 't126', 't127', 't128', 't129', 't130', 't131', 't132', 't133', 't134', 't135', 't136',

In [3]:
# === Encode labels ===
label_encoder = LabelEncoder()
temporal_df['Label_Encoded'] = label_encoder.fit_transform(temporal_df['Defect_Label'])

# Reshape temporal data: 500 = 100 timesteps × 5 features
TIME_STEPS = 100
NUM_FEATURES = 5
temporal_cols = [f't{i}' for i in range(500)]
temporal_values = temporal_df[temporal_cols].values[:1000]  # take only 1000 segments
temporal_reshaped = temporal_values.reshape(-1, TIME_STEPS, NUM_FEATURES)

In [4]:
# Normalize spatial features: ['Thickness_mm', 'Temperature_C']
spatial_features_df = spatial_df[['Segment_ID', 'Thickness_mm', 'Temperature_C']].copy()
scaler = StandardScaler()
spatial_features_df[['Thickness_mm', 'Temperature_C']] = scaler.fit_transform(
    spatial_features_df[['Thickness_mm', 'Temperature_C']]
)

In [5]:
# Merge spatial features into temporal data
merged_spatial = temporal_df[['Segment_ID']].merge(spatial_features_df, on='Segment_ID', how='left')
spatial_features = merged_spatial[['Thickness_mm', 'Temperature_C']].values


In [6]:
# Extract encoded labels
labels = temporal_df['Label_Encoded'].values

# === Train-test split ===
X_temp_train, X_temp_test, X_spat_train, X_spat_test, y_train, y_test = train_test_split(
    temporal_reshaped, spatial_features, labels, test_size=0.2, stratify=labels, random_state=42
)

In [7]:
# === Define Dual-Branch Model ===
temporal_input = Input(shape=(TIME_STEPS, NUM_FEATURES), name='Temporal_Input')
x1 = LSTM(64, return_sequences=False)(temporal_input)
x1 = Dropout(0.3)(x1)

spatial_input = Input(shape=(spatial_features.shape[1],), name='Spatial_Input')
x2 = Dense(32, activation='relu')(spatial_input)
x2 = Dropout(0.3)(x2)

In [8]:
combined = Concatenate()([x1, x2])
x = Dense(32, activation='relu')(combined)
output = Dense(len(np.unique(labels)), activation='softmax')(x)

model = Model(inputs=[temporal_input, spatial_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
from sklearn.metrics import classification_report

# === Train ===
model.fit(
    [X_temp_train, X_spat_train], y_train,
    validation_data=([X_temp_test, X_spat_test], y_test),
    epochs=20, batch_size=16, verbose=2
)

# === Evaluate ===
loss, accuracy = model.evaluate([X_temp_test, X_spat_test], y_test, verbose=0)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")

# === Classification Report ===
y_pred = model.predict([X_temp_test, X_spat_test])
y_pred_labels = np.argmax(y_pred, axis=1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_labels, target_names=label_encoder.classes_))

Epoch 1/20
50/50 - 0s - 9ms/step - accuracy: 0.7575 - loss: 0.3529 - val_accuracy: 0.7200 - val_loss: 0.3514
Epoch 2/20
50/50 - 0s - 9ms/step - accuracy: 0.7525 - loss: 0.3555 - val_accuracy: 0.7150 - val_loss: 0.3528
Epoch 3/20
50/50 - 0s - 9ms/step - accuracy: 0.7600 - loss: 0.3550 - val_accuracy: 0.7600 - val_loss: 0.3525
Epoch 4/20
50/50 - 0s - 9ms/step - accuracy: 0.7225 - loss: 0.3613 - val_accuracy: 0.7550 - val_loss: 0.3521
Epoch 5/20
50/50 - 0s - 9ms/step - accuracy: 0.7487 - loss: 0.3528 - val_accuracy: 0.7400 - val_loss: 0.3514
Epoch 6/20
50/50 - 0s - 9ms/step - accuracy: 0.7513 - loss: 0.3559 - val_accuracy: 0.7250 - val_loss: 0.3511
Epoch 7/20
50/50 - 0s - 9ms/step - accuracy: 0.7588 - loss: 0.3542 - val_accuracy: 0.7300 - val_loss: 0.3510
Epoch 8/20
50/50 - 0s - 9ms/step - accuracy: 0.7563 - loss: 0.3531 - val_accuracy: 0.7500 - val_loss: 0.3547
Epoch 9/20
50/50 - 0s - 9ms/step - accuracy: 0.7613 - loss: 0.3504 - val_accuracy: 0.7450 - val_loss: 0.3567
Epoch 10/20
50/50 -